# Find your soulmate

I created this little notebook during a Data Science bootcamp. One day we were jokingly asking each other "light-hearted" preference questions like 'are you a cat or dog person' or 'pineapple on pizza yes/no' and I decided to set up a small google survey and wrote this notebook so that we could find out our bootcamp soulmates. 

Calculates the number of matching answers between participants and adds the most similar participant(s) as last column.  
Download results of survey as data.csv

In [31]:
import pandas as pd
import numpy as np

In [32]:
df = pd.read_csv("data/data.csv")

In [33]:
df = df.drop("Timestamp",errors="ignore",axis=1)
col = df.pop("Name:")
df.insert(0,"name",col) # Name as first column

In [34]:
df = df.set_index(df.name)

In [35]:
zero_arr = np.zeros((df.shape[0],df.shape[0]), int) # Get a n x n matrix where n = number of participants in the survey

mat = pd.DataFrame(zero_arr, columns=df.name,index=df.name) # Create the n x n matrix as df with names' as columns.


In [36]:
for name1 in df.name:
    matches = []
    for name2 in df.name:
        matches.append(sum(df.loc[name1] == df.loc[name2]))
    mat.loc[name1] = matches  # Counts number of matches row by row.

In [37]:
np.fill_diagonal(mat.values,0) # Set diagonal to 0 so that max() works.

In [38]:
mat.loc["Jan"]

name
Jan        0
Shahi      3
Larissa    4
Ana        7
Grace      5
Name: Jan, dtype: int64

In [46]:
mat.columns[mat.loc["Jan"] == mat.loc["Jan"].max()]

Index(['Ana'], dtype='object', name='name')

In [29]:
max_li = []
for name in mat.columns:
    mat.loc[name] == mat.loc[name].max()
    max_li.append(mat.columns[mat.loc[name] == mat.loc[name].max()].tolist()) # Selects those columns(=names) with Max value in row.
mat["best_match"] = max_li

# Loop instead of idmax() so that ties can be seen.

In [ ]:
mat